In [1]:
# !pip install obspy pyproj matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 2.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4


In [2]:
try:
    import obspy

    obspy.read()
except TypeError:
    # Needs to restart the runtime once, because obspy only works properly after restart.
    print(
        "Stopping RUNTIME. If you run this code for the first time, this is expected. Colaboratory will restart automatically. Please run again."
    )
    exit()

In [3]:
import os

import matplotlib.pyplot as plt
import numpy as np
import obspy
from obspy import UTCDateTime, read
from obspy.clients.fdsn import Client
from obspy.signal.tf_misfit import cwt

In [ ]:
client = Client("IRIS")

t0 = [UTCDateTime("2023/09/16 10:15:00")]

secin1hour = 3600

daynum = 7

for it in t0:
    t1 = it
    t2 = t1 + secin1hour * 24 * daynum

    net = "SG"
    sta = "MEMB"
    loc = "--"
    cha = "LGZ"

    out = "%s.%s.%s.mseed" % (t1.strftime("%Y-%m-%d"), net, sta)
    if os.path.isfile(out):
        stream = read(out)
    else:
        stream = client.get_waveforms(
            network=net,
            station=sta,
            location=loc,
            channel=cha,
            starttime=t1,
            endtime=t2,
        )
        stream.write(out)

    stream.merge()
    print(stream)
    stream.decimate(10)
    #     stream.trim(t1, t1+86400)

    # stream.plot()
    tr = stream[0]

    npts = tr.stats.npts
    dt = tr.stats.delta
    t = np.linspace(0, dt * npts, npts) / (3600 * 24)
    f_min = 5e-3
    f_max = 25e-3

    tr.detrend("linear")
    tr.taper(max_percentage=0.01)
    tr.filter("bandpass", freqmin=1e-3, freqmax=40e-3, corners=3, zerophase=True)

    scalogram = cwt(tr.data, dt, 70, f_min, f_max, nf=350)

    x, y = np.meshgrid(
        t, np.logspace(np.log10(f_min), np.log10(f_max), scalogram.shape[0])
    )

    fig = plt.figure(figsize=(25, 6))
    ax0 = fig.add_subplot(111)
    im = ax0.pcolormesh(
        x,
        y * 1000,
        np.log10(np.abs(scalogram)),
        cmap="inferno",
        vmin=2,
        vmax=4,
        rasterized=True,
    )
    # plt.plot([t[0], t[-1]],[10.88, 10.88],'g:')
    # plt.plot([t[0], t[-1]],[10.88+0.022, 10.88+0.022],'g--')
    # plt.plot([t[0], t[-1]],[15.7e-3, 15.7e-3],'b--')
    # plt.plot([t[0], t[-1]],[17.5e-3, 17.5e-3],'b--')
    fig.colorbar(im, ax=ax0, label="log10(amplitude)")
    ax0.set_xlabel("Time after %s [days]" % tr.stats.starttime)
    ax0.set_ylabel("Frequency [mHz]")
    ax0.set_title(
        "%s-%s-%s-%s"
        % (tr.stats.network, tr.stats.station, tr.stats.location, tr.stats.channel)
    )
    # ax0.set_yscale('log')
    ax0.set_ylim(f_min * 1000, f_max * 1000)
    plt.savefig(
        "SGMEMB_VLP_Spectro_ %s.png" % tr.stats.starttime.strftime("%Y-%m-%d"), dpi=150
    )
    plt.savefig("SGMEMB_VLP_Spectro_%s.pdf" % tr.stats.starttime.strftime("%Y-%m-%d"))
    plt.show()

    del stream
    del tr
    del scalogram
    del x
    del y
    del t
    del fig